***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [2]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE  SCHEDULED_STOP
cluster-0016  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import json
import hashlib

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 13 11:11 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [7]:
spark

In [8]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ir-proj-bucket'
BUCKET_NAME = 'ir-proj-bucket'
full_path = f"gs://{bucket_name}/"
paths = []

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith(".parquet"):
        paths.append(full_path + b.name)

print(len(paths), "parquet files found")

60 parquet files found


***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [9]:
parquetFile = spark.read.parquet(*paths)
wiki_data_body = parquetFile.select("text","id").rdd
wiki_data_anchor = parquetFile.select("anchor_text","id").rdd
wiki_data_title = parquetFile.select("title","id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [10]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [11]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [13]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS
def flatten_lst(t):
    return [item for sublist in t for item in sublist]

def tokenize(text):
    tokens = []
    for m in RE_WORD.finditer(text.lower()):
        w = m.group()
        if w not in all_stopwords:
            tokens.append(w)
    return tokens

def word_count(text, doc_id):
    """
    """
    if text is None:
        return []
    # tokenization by regex
    tokens = [m.group() for m in RE_WORD.finditer(text.lower())]
    # remove stopwords
    tokens = [t for t in tokens if t not in all_stopwords]

    return [(t, doc_id) for t in tokens]

def word_count_anchor(id,list_of_tupels):
    t=flatten_lst([RE_WORD.finditer(text.lower()) for dest_id,text in list_of_tupels])

    tokens = [token.group() for token in t]

    terms = {}
    tf = Counter()
    for token in tokens:
        if token in all_stopwords:
            continue
        current = tf.get(token, 0)
        tf[token] = current + 1
        terms[token] = (id,current +1)
    return list(terms.items())

def reduce_word_counts(doc_ids_iter):
    cnt = Counter(doc_ids_iter)      # Counter: doc_id -> tf
    return sorted(cnt.items())       # [(doc_id, tf), ...]


def calculate_df(postings_rdd):
    return postings_rdd.mapValues(lambda pl: len(pl))


def partition_postings_and_write(postings_rdd, base_dir): # הוספנו base_dir
    # (term, pl) -> (bucket_id, (term, pl))
    bucketed = postings_rdd.map(
        lambda x: (token2bucket_id(x[0]), (x[0], x[1]))
    )

    # groupBy bucket_id → (bucket_id, iterable[(term, pl)])
    grouped = bucketed.groupByKey()
    
    # התיקון: אנחנו מעבירים את base_dir ואת BUCKET_NAME לפונקציה של האינדקס
    # וודא ש-BUCKET_NAME מוגדר אצלך כ-'ir-proj-bucket'
    return grouped.map(lambda bwpl: InvertedIndex.write_a_posting_list(bwpl, base_dir, BUCKET_NAME))


In [15]:
def build_index_ver1(rdd_data,location):
    word_counts = rdd_data.flatMap(lambda x: word_count(x[0], x[1]))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)

    #w2df = calculate_df(postings)
    postings_filtered = postings.filter(lambda x: len(x[1]) > 50)
    w2df_dict = calculate_df(postings_filtered).collectAsMap()
    posting_locs_list = partition_postings_and_write(postings_filtered, location).collect()


    super_posting_locs = defaultdict(list)
    for posting_loc in posting_locs_list:
      for k, v in posting_loc.items():
        super_posting_locs[k].extend(v)

    # Create inverted index instance
    inverted = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df_dict
    # write the global stats out

    inverted.write_index(location, 'index', bucket_name=BUCKET_NAME)

    return inverted


In [16]:
def build_index_ver2(rdd_data,location):
    word_counts_body = rdd_data.flatMap(lambda x: word_count_anchor(x[0], x[1]))
    postings = word_counts_body.groupByKey().mapValues(reduce_word_counts)

    w2df = calculate_df(postings)

    w2df_dict = w2df.collectAsMap()
    posting_locs_list = partition_postings_and_write(postings,location).collect()

    super_posting_locs = defaultdict(list)
    for posting_loc in posting_locs_list:
      for k, v in posting_loc.items():
        super_posting_locs[k].extend(v)

    # Create inverted index instance
    inverted = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df_dict
    # write the global stats out
    inverted.write_index(location, 'index')

    return inverted

In [15]:
index_title=build_index_ver1(wiki_data_title,'./title_index')

In [ ]:
index_body=build_index_ver1(wiki_data_body,'./body_index')

In [36]:
index_anchor=build_index_ver2(wiki_data_anchor,'./anchor_index')#########################################################

hii


In [17]:
def build_and_save_DL(wiki_rdd, out_path, tokenize_fn):
    DL = (
        wiki_rdd
        .map(lambda x: (x[1], len(tokenize_fn(x[0]))))
        .collectAsMap()
    )
    with open(out_path, "w") as f:
        json.dump(DL, f)
    return len(DL)


In [18]:
num = build_and_save_DL(wiki_data_title, "title_index/title_DL.json", tokenize)
print(num) #sanity check


6348910


In [ ]:
num = build_and_save_DL(wiki_data_body, "body_index/body_DL.json", tokenize)
print(num) #sanity check

6348910


In [ ]:
# wiki_data_anchor: RDD של Rows, שלכל אחד יש .anchor_text = list[Row(id, text)]
anchor_text_per_doc = (
    wiki_data_anchor
    .flatMap(lambda row: row.anchor_text)                 # Row(id=..., text=...)
    .map(lambda r: (r.id, r.text))                        # (doc_id, anchor_text)
    .groupByKey()
    .mapValues(lambda texts: " ".join(texts))             # (doc_id, "all anchors together")
    .map(lambda x: (x[1], x[0]))                          # (text, doc_id) -> מתאים לפונקציה שלך
)

num = build_and_save_DL(anchor_text_per_doc, "anchor_index/anchor_DL.json", tokenize)
print(num)


5867103


In [19]:
import os, json

def build_and_save_doc2title_from_parquet(parquet_df, out_dir, num_buck=800):
    os.makedirs(out_dir, exist_ok=True)

    # (bucket, [id,title])
    buckets = (
        parquet_df.select("id", "title")
        .rdd
        .map(lambda r: (int(r["id"]) % num_buck, [int(r["id"]), str(r["title"])]))
        .groupByKey()
    )

    for k, pairs in buckets.toLocalIterator():
        with open(os.path.join(out_dir, f"d2t{k}.json"), "w", encoding="utf-8") as f:
            json.dump(list(pairs), f, ensure_ascii=False)

    return num_buck

# יצירה לתיקייה מקומית ליד הקוד (כמו שה-getTitle מצפה)
build_and_save_doc2title_from_parquet(parquetFile, out_dir="./doc2title", num_buck=800)


800

In [27]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def word_count_stem(text, doc_id):
    if text is None:
        return []
    tokens = [m.group() for m in RE_WORD.finditer(text.lower())]
    tokens = [t for t in tokens if t not in all_stopwords]
    tokens = [stemmer.stem(t) for t in tokens]
    return [(t, doc_id) for t in tokens]


def build_index_ver1_stemm(rdd_data,location):
    word_counts = rdd_data.flatMap(lambda x: word_count_stem(x[0], x[1]))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)

    #w2df = calculate_df(postings)
    postings_filtered = postings.filter(lambda x: len(x[1]) > 50)
    w2df_dict = calculate_df(postings_filtered).collectAsMap()
    posting_locs_list = partition_postings_and_write(postings_filtered, location).collect()


    super_posting_locs = defaultdict(list)
    for posting_loc in posting_locs_list:
      for k, v in posting_loc.items():
        super_posting_locs[k].extend(v)

    # Create inverted index instance
    inverted = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df_dict
    # write the global stats out

    inverted.write_index(location, 'index', bucket_name=BUCKET_NAME)

    return inverted


In [ ]:
index_body_stem = build_index_ver1_stemm(wiki_data_body, './body_index_stem')

In [30]:
index_title_stem = build_index_ver1_stemm(wiki_data_title, './title_index_stem')


In [ ]:
index_anchor_stem = build_index_ver1_stemm(wiki_data_anchor, './anchor_index_stem')


In [36]:
!mkdir -p /home/dataproc/body_index_stem



In [37]:
!mkdir -p /home/dataproc/title_index_stem

In [47]:
!gsutil -m rsync -r gs://ir-proj-bucket/body_index_stem /home/dataproc/body_index_stem




Building synchronization state...
Starting synchronization...


In [40]:
!gsutil -m cp -r gs://ir-proj-bucket/title_index_stem/* /home/dataproc/title_index_stem/


Copying gs://ir-proj-bucket/title_index_stem/0_000.bin...
Copying gs://ir-proj-bucket/title_index_stem/0_posting_locs.pickle...           
Copying gs://ir-proj-bucket/title_index_stem/100_000.bin...                     
Copying gs://ir-proj-bucket/title_index_stem/101_000.bin...                     
Copying gs://ir-proj-bucket/title_index_stem/100_posting_locs.pickle...         
Copying gs://ir-proj-bucket/title_index_stem/102_posting_locs.pickle...         
Copying gs://ir-proj-bucket/title_index_stem/104_posting_locs.pickle...         
Copying gs://ir-proj-bucket/title_index_stem/101_posting_locs.pickle...
Copying gs://ir-proj-bucket/title_index_stem/103_posting_locs.pickle...         
Copying gs://ir-proj-bucket/title_index_stem/105_posting_locs.pickle...
Copying gs://ir-proj-bucket/title_index_stem/102_000.bin...                     
Copying gs://ir-proj-bucket/title_index_stem/103_000.bin...                     
Copying gs://ir-proj-bucket/title_index_stem/106_000.bin...          

/home/dataproc
d2t0.json
d2t1.json
d2t10.json
d2t100.json
d2t101.json
d2t102.json
d2t103.json
d2t104.json
d2t105.json
d2t106.json


In [52]:
!find /home/dataproc/body_index_stem -type f | wc -l




3113


In [46]:
print("hi")

hi


In [ ]:
# # time the index creation time
# t_start = time()
# # word counts map
# word_counts = wiki_data_body.flatMap(lambda x: word_count(x[0], x[1]))
# postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# # filtering postings and calculate df
# postings_filtered = postings.filter(lambda x: len(x[1])>50)
# w2df = calculate_df(postings_filtered)
# w2df_dict = w2df.collectAsMap()
# # partition posting lists and write out
# _ = partition_postings_and_write(postings_filtered).collect()
# index_const_time = time() - t_start

t_start = time()

# 1. word counts map
word_counts = wiki_data_body.flatMap(lambda x: word_count(x[0], x[1]))

# 2. groupByKey 
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1]) > 50)

w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

# 4. כתיבת ה-Postings לענן - שים לב לשינוי כאן!
# הוספנו את 'body_index' כפרמטר השני
_ = partition_postings_and_write(postings_filtered, 'body_index').collect()

index_const_time = time() - t_start
print(f"Index created and saved to body_index in {index_const_time:.2f} seconds")

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = wiki_data_title.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

In [ ]:
anchor

In [ ]:
# test index construction time
assert index_const_time < 60*120

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.4 MiB/ 18.4 MiB]                                                
Operation completed over 1 objects/18.4 MiB.                                     


In [ ]:
!gsutil ls -lh $index_dst

 18.45 MiB  2025-12-11T19:05:55Z  gs://bars-wiki-bucket-2025/postings_gcp/index.pkl
TOTAL: 1 objects, 19343319 bytes (18.45 MiB)


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
# Put your `generate_graph` function here
def generate_graph(page):
    kv = page.map(lambda row: (row["id"], row["anchor_text"]))
    temp = kv.partitionBy(16)

    #  (src -> dst)
    def edges_from_item(item):
        src_id, anchors = item
        if anchors is None:
            return []
        return [
            (src_id, a[0])
            for a in anchors
            if a is not None and len(a) > 0 and a[0] is not None
        ]

    edges = temp.flatMap(edges_from_item).distinct()

    # (vertices)
    def dst_vertices(item):
        _, anchors = item
        if anchors is None:
            return []
        return [
            (a[0],)
            for a in anchors
            if a is not None and len(a) > 0 and a[0] is not None
        ]

    p1 = temp.flatMap(dst_vertices)     
    p2 = temp.map(lambda x: (x[0],))     
    vert = p1.union(p2).distinct()

    return edges, vert

In [ ]:
t_start = time()
pages_links = parquetFile.select("id", "anchor_text").rdd
print(pages_links)
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

MapPartitionsRDD[32] at javaToPython at NativeMethodAccessorImpl.java:0


+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750| 9917.889528786589|
|  10568| 5387.609470096125|
|  32927| 5284.298441243094|
|  30680| 5130.386005836191|
|5843419| 4959.648354746549|
|  68253| 4771.279909629092|
|  31717| 4488.233081199124|
|  11867| 4148.154882176614|
|  14533|3998.1437395714415|
| 645042| 3533.109297540147|
|  17867|3247.4607632514617|
|5042916| 2993.201445108151|
|4689264|2983.5765511544078|
|  14532|2935.9785290242507|
|  25391| 2904.764828582977|
|   5405|2892.6298433630327|
|4764461|2835.5565581932306|
|  15573| 2785.033493615905|
|   9316|2783.2072557286933|
|8569916| 2776.450966760947|
+-------+------------------+
only showing top 20 rows



In [33]:
# test that PageRank computaion took less than 2 hours
assert pr_time < 60*120
print(pr_time)

6921.575781106949


# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [30]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [31]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://bars-wiki-bucket-2025/postings_gcp


In [1]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 9.74
print(f'I used {cost} USD credit during the course of this assignment')

I used 9.74 USD credit during the course of this assignment


**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [27]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = True # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE
temp = pages_links.partitionBy(16) #partitioning for speed
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
edges = temp.flatMap(lambda x:[(x[0],each[0]) for each in x[1]]).distinct() 
p1 = temp.flatMap(lambda x:[(each[0],) for each in x[1]])
p2 = temp.map(lambda x:(x[0],))
vert = p1.union(p2).distinct()

pr_time_Bonus = time() - t_start


In [34]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag